In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl

import numpy as np
import os
import pandas as pd
import pickle
import re
import seaborn as sns

plt.style.use('./stylefiles/plot.mplstyle')
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10

In [2]:
PATH = './results/base/'

In [3]:
# List the files
results_folders = os.listdir(PATH)

# Load the data
list_results = []
for folder in results_folders:
    if folder[0] == '.':
        continue
    folder_split = re.split("_", folder)
    N = int(folder_split[0][1:])
    M = folder_split[1][1:]
    
    files = os.listdir(PATH + folder)
    list_ISE = len(files) * [None]
    for idx, file in enumerate(files):
        with open(PATH + folder + "/" + file, "rb") as f:
            data = pickle.load(f)
        errors = data['errors']
        ISE = {
            'ISEcov': errors['ISE_cov'],
            'ISEpsplines': errors['ISE_psplines'],
            'ISEgram': errors['ISE_gram']
        }
        ISE = pd.DataFrame.from_records(ISE).reset_index()
        ISE.insert(0, "N", N)
        ISE.insert(0, "M", M)
        list_ISE[idx] = ISE
    list_results.append(pd.concat(list_ISE))
results = pd.concat(list_results)

In [4]:
SORT_VALUES = ['11-11-21', '26-26-51', '101-51-201']

results_pp = pd.melt(
    results,
    id_vars=['index', 'M', 'N'],
    value_vars=['ISEcov', 'ISEpsplines', 'ISEgram']
)
results_pp['index'] = results_pp['index'] + 1
results_pp.M = results_pp.M.astype("category")
results_pp.M = results_pp.M.cat.set_categories(SORT_VALUES)
results_pp = results_pp.sort_values(by=['N', 'M'])

In [5]:
gg = sns.catplot(
    data=results_pp,
    x="index", y="value", hue="variable",
    col="N", row="M",
    kind="box",
    flierprops=dict(marker="+", markerfacecolor="gray", markersize=1),
    height=2,
    aspect=2
)
gg.set_titles(template="$N = {col_name} \:|\: M = {row_name}$", size=12)
gg.set_xlabels("Eigenfunctions", fontsize=10)
gg.set_ylabels("ISE (log scale)", fontsize=10)
gg.set(ylim=(1e-3, 10))
for ax in gg.axes.flat:
    ax.set_yscale("log")
gg.fig.tight_layout()

sns.move_legend(
    gg, "center",
    bbox_to_anchor=(.5, 0), ncol=3,
    labels=["(Tensor) PCA", "2D/1D B-Splines", "Gram"],
    title=None, frameon=False, fontsize=12
)

plt.savefig(
    f'ISE.eps',
    format='eps',
    bbox_inches='tight'
)
plt.close()